In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, mixed_precision
from tensorflow.keras.applications.resnet50 import preprocess_input
from utils.DenseCL import DenseCL
from utils.EpochLogger import EpochLogger

2026-02-12 12:00:20.632649: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-12 12:00:20.708100: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-12 12:00:22.163227: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
tf.keras.backend.clear_session()

gpus = tf.config.list_physical_devices("GPU")
for g in gpus:
    tf.config.experimental.set_memory_growth(g, True)

mixed_precision.set_global_policy("mixed_float16" if gpus else "float32")

if len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy()
elif len(gpus) == 1:
    strategy = tf.distribute.OneDeviceStrategy("/GPU:0")
else:
    strategy = tf.distribute.OneDeviceStrategy("/CPU:0")

print("GPUs:", gpus, "replicas:", strategy.num_replicas_in_sync)


GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] replicas: 1


I0000 00:00:1770922824.996107   22253 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2252 MB memory:  -> device: 0, name: NVIDIA T550 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 7.5


In [3]:
IMAGE_SIZE = 224
BATCH_SIZE = 8
EPOCHS = 200
STEPS_PER_EPOCH = 8163 // BATCH_SIZE  # There are 8163 images in the training set
QUEUE_SIZE = 4096  # Size of the negative sample queue for contrastive learning

with strategy.scope():
    backbone = tf.keras.applications.ResNet50(
        include_top=False, weights="imagenet", input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)
    )

    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=1e-3,
        decay_steps=STEPS_PER_EPOCH * EPOCHS,
        alpha=0.1
    )

    optimizer = tf.keras.optimizers.SGD(
        learning_rate=lr_schedule, 
        momentum=0.9,
        weight_decay=0.0001
    )

    model = DenseCL(backbone, queue_size=QUEUE_SIZE)
    model.compile(optimizer=optimizer, run_eagerly=False)

2026-02-12 12:00:28.695333: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


In [4]:
aug_q = tf.keras.Sequential([
    layers.RandomFlip("horizontal", seed=1),
    layers.RandomRotation(0.2, seed=2),
    layers.RandomZoom(0.3, seed=3),
    layers.RandomContrast(0.2, seed=4),
    layers.RandomBrightness(0.2, seed=5),
    layers.GaussianNoise(0.1, seed=6),
])

aug_k = tf.keras.Sequential([
    layers.RandomFlip("horizontal", seed=7),
    layers.RandomRotation(0.2, seed=8),
    layers.RandomZoom(0.3, seed=9),
    layers.RandomContrast(0.2, seed=10),
    layers.RandomBrightness(0.2, seed=11),
    layers.GaussianNoise(0.1, seed=12),
])

def make_two_views(image):
    image = tf.cast(image, tf.float32)
    x_q = aug_q(image, training=True)
    x_k = aug_k(image, training=True)
    return x_q, x_k

def preprocess_views(x_q, x_k):
    return preprocess_input(x_q), preprocess_input(x_k)


In [ ]:
images_dir = "/mnt/h/figures_non_labeled" # Update this path to your actual image directory if needed

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    images_dir,
    label_mode=None,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
)

train_ds = (
    train_ds
    .map(make_two_views, num_parallel_calls=tf.data.AUTOTUNE)
    .map(preprocess_views, num_parallel_calls=tf.data.AUTOTUNE)
    .ignore_errors() 
    .repeat()
    .prefetch(tf.data.AUTOTUNE)
)

Found 7 files.


In [7]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir="logs/densecl_test"), EpochLogger()],
)

model.backbone.save(f"trained_models/densecl_resnet50_{EPOCHS}.keras")

Epoch 1/200


2026-02-12 12:00:54.610785: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91900


 514/1020 ━━━━━━━━━━━━━━━━━━━━ 7:26 882ms/step - dense: 5.2916 - global: 1.9245 - loss: 1.9245

KeyboardInterrupt: 